In [3]:
import os
import sys
import math
import shutil
import glob
from pathlib import Path
from dotenv import load_dotenv
from openai import OpenAI
from pydub import AudioSegment

# --- Constants ---
MAX_CHUNK_SIZE_BYTES = 24.5 * 1024 * 1024  # 24.5 MB to be safe under OpenAI's 25MB limit
TARGET_BITRATE_KBPS = "192k" # Target bitrate for MP3 chunk export and size estimation
UPLOADS_DIR = "uploads"
TRANSCRIPTIONS_DIR = "transcriptions"
TEMP_CHUNK_SUBDIR = "temp_audio_chunks" # Relative to project root
ALLOWED_EXTENSIONS = [".m4a", ".mp3"]

# --- Helper Function ---
def estimate_segment_duration_ms(audio_duration_ms, audio_channels, target_bitrate_kbps_str, max_size_bytes):
    """
    Estimates the duration (in ms) for audio segments to keep them under max_size_bytes
    when exported as MP3 at target_bitrate_kbps.
    """
    try:
        target_bitrate_kbps = int(target_bitrate_kbps_str.replace('k', ''))
    except ValueError:
        print(f"Error: Invalid target bitrate format: {target_bitrate_kbps_str}. Expected format like '192k'.")
        return -1 # Indicate error

    if target_bitrate_kbps <= 0:
        print(f"Error: Target bitrate must be positive. Got: {target_bitrate_kbps_str}")
        return -1
        
    bytes_per_second_at_target_bitrate = (target_bitrate_kbps * 1000) / 8
    
    if bytes_per_second_at_target_bitrate <= 0:
        print(f"Error: Calculated bytes_per_second_at_target_bitrate is not positive. Check target_bitrate_kbps.")
        return -1

    max_duration_seconds_for_chunk = max_size_bytes / bytes_per_second_at_target_bitrate
    estimated_chunk_duration_ms = math.floor(max_duration_seconds_for_chunk * 1000)
    
    return min(estimated_chunk_duration_ms, audio_duration_ms)

# --- Main Processing Logic ---
def process_latest_audio():
    print("Starting transcription process...")
    print(f"NOTE: This script relies on FFmpeg. If you encounter errors related to 'ffmpeg not found', please ensure FFmpeg is installed and in your system's PATH.\n")

    # Create necessary directories
    os.makedirs(UPLOADS_DIR, exist_ok=True)
    os.makedirs(TRANSCRIPTIONS_DIR, exist_ok=True)
    
    # Load environment variables
    load_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")

    if not api_key:
        print("Error: OPENAI_API_KEY not found in .env file or environment variables.")
        print("Please create a .env file at the project root with your API key.")
        return

    client = OpenAI(api_key=api_key)

    # Find the most recent audio file
    upload_path = Path(UPLOADS_DIR)
    audio_files = []
    for ext in ALLOWED_EXTENSIONS:
        audio_files.extend(upload_path.glob(f'*{ext}'))
        audio_files.extend(upload_path.glob(f'*{ext.upper()}')) # Also check uppercase extensions

    if not audio_files:
        print(f"No audio files ({', '.join(ALLOWED_EXTENSIONS)}) found in '{UPLOADS_DIR}'.")
        return

    try:
        latest_file = max(audio_files, key=lambda p: p.stat().st_mtime)
    except Exception as e:
        print(f"Error finding latest file: {e}")
        return
        
    audio_filename = latest_file.name
    input_file_path = str(latest_file)
    print(f"Processing most recent file: {audio_filename}")

    base_name, input_ext_with_dot = os.path.splitext(audio_filename)
    input_ext = input_ext_with_dot.lower()

    if input_ext not in ALLOWED_EXTENSIONS:
        print(f"Error: File '{audio_filename}' has an unsupported extension '{input_ext}'. This should not happen if glob pattern is correct.")
        return
    
    audio_format = input_ext[1:] # Remove leading dot for pydub

    output_filename = base_name + ".txt"
    output_file_path = os.path.join(TRANSCRIPTIONS_DIR, output_filename)

    # Validate input file (though already found by glob)
    if not os.path.exists(input_file_path):
        print(f"Error: Input file not found: {input_file_path} (this is unexpected)")
        return
    if os.path.getsize(input_file_path) == 0:
        print(f"Error: Input file is empty: {input_file_path}")
        return

    # Manage temporary chunk directory
    temp_chunk_path = Path(TEMP_CHUNK_SUBDIR)
    if temp_chunk_path.exists():
        shutil.rmtree(temp_chunk_path)
    os.makedirs(temp_chunk_path, exist_ok=True)

    transcribed_texts = []
    chunk_files_paths = [] # Store paths of created chunks

    try:
        print(f"Loading audio file: {input_file_path}...")
        audio = AudioSegment.from_file(input_file_path, format=audio_format)
        
        audio_duration_ms = len(audio)
        audio_channels = audio.channels
        
        segment_duration_ms = estimate_segment_duration_ms(
            audio_duration_ms, 
            audio_channels, 
            TARGET_BITRATE_KBPS, 
            MAX_CHUNK_SIZE_BYTES
        )

        if segment_duration_ms == -1: # Error in estimation
            return # Error message already printed by helper
        if segment_duration_ms <= 0:
            print("Error: Calculated segment duration is zero or negative. This might be due to a very small or problematic audio file, or an issue with bitrate estimation.")
            return

        num_chunks = math.ceil(audio_duration_ms / segment_duration_ms)
        print(f"Splitting into {num_chunks} chunks (estimated)..." if num_chunks > 1 else "Processing as a single chunk...")

        for i in range(num_chunks):
            start_ms = i * segment_duration_ms
            end_ms = min((i + 1) * segment_duration_ms, audio_duration_ms)
            if start_ms >= end_ms: 
                continue
            chunk = audio[start_ms:end_ms]
            
            chunk_file_name = f"chunk_{i:03d}.mp3"
            chunk_file_path_obj = temp_chunk_path / chunk_file_name
            
            print(f"Exporting chunk {i+1}/{num_chunks} to {chunk_file_path_obj}...")
            chunk.export(str(chunk_file_path_obj), format="mp3", bitrate=TARGET_BITRATE_KBPS)
            
            actual_chunk_size = os.path.getsize(str(chunk_file_path_obj))
            if actual_chunk_size > MAX_CHUNK_SIZE_BYTES * 1.1: 
                 print(f"Warning: Chunk {chunk_file_path_obj} size ({actual_chunk_size / (1024*1024):.2f}MB) is larger than expected. Transcription might fail.")

            chunk_files_paths.append(str(chunk_file_path_obj))

        if not chunk_files_paths:
            print("No audio chunks were generated. Check if the audio file is too short or empty.")
            return

        print(f"\nStarting transcription for {len(chunk_files_paths)} chunks...")
        for i, chunk_fp_str in enumerate(chunk_files_paths):
            print(f"Transcribing slice {i+1}/{len(chunk_files_paths)} ({os.path.basename(chunk_fp_str)})...")
            try:
                with open(chunk_fp_str, "rb") as audio_file_handle:
                    transcript = client.audio.transcriptions.create(
                        model="whisper-1",
                        file=audio_file_handle
                    )
                transcribed_texts.append(transcript.text.strip())
            except Exception as e:
                print(f"Error transcribing chunk {chunk_fp_str}: {e}")
                transcribed_texts.append(f"[Error transcribing {os.path.basename(chunk_fp_str)}: {str(e)}]")

        print(f"\nWriting transcript to: {output_file_path}...")
        full_transcript = "\n\n".join(transcribed_texts)
        with open(output_file_path, "w", encoding="utf-8") as f:
            f.write(full_transcript)
        
        print(f"Transcript successfully written to {output_file_path}")
        print(f"\n--- Transcript for {audio_filename} ---")
        print(full_transcript)
        print("--- End of Transcript ---")

    except FileNotFoundError as fnf_error:
        if 'ffmpeg' in str(fnf_error).lower() or 'avprobe' in str(fnf_error).lower() or 'avconv' in str(fnf_error).lower():
             print(f"\nERROR: FFmpeg (or its components like avprobe/avconv) not found. \npydub requires FFmpeg to be installed and in your system's PATH for M4A/MP3 processing. \nPlease install FFmpeg and ensure it's in your PATH.")
        else:
            print(f"A file-related error occurred: {fnf_error}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        if temp_chunk_path.exists():
            print("Cleaning up temporary files...")
            shutil.rmtree(temp_chunk_path)
    print("\nTranscription process finished.")

# --- Execute the main function ---
process_latest_audio()


Starting transcription process...
NOTE: This script relies on FFmpeg. If you encounter errors related to 'ffmpeg not found', please ensure FFmpeg is installed and in your system's PATH.

Processing most recent file: srp3.mp3
Loading audio file: uploads\srp3.mp3...
Splitting into 8 chunks (estimated)...
Exporting chunk 1/8 to temp_audio_chunks\chunk_000.mp3...
Exporting chunk 2/8 to temp_audio_chunks\chunk_001.mp3...
Exporting chunk 3/8 to temp_audio_chunks\chunk_002.mp3...
Exporting chunk 4/8 to temp_audio_chunks\chunk_003.mp3...
Exporting chunk 5/8 to temp_audio_chunks\chunk_004.mp3...
Exporting chunk 6/8 to temp_audio_chunks\chunk_005.mp3...
Exporting chunk 7/8 to temp_audio_chunks\chunk_006.mp3...
Exporting chunk 8/8 to temp_audio_chunks\chunk_007.mp3...

Starting transcription for 8 chunks...
Transcribing slice 1/8 (chunk_000.mp3)...
Transcribing slice 2/8 (chunk_001.mp3)...
Transcribing slice 3/8 (chunk_002.mp3)...
Transcribing slice 4/8 (chunk_003.mp3)...
Transcribing slice 5/8 (